In [1]:
import pandas
column_name = ['type', 'series', 'name', 'code']
column_name.extend([str(i) for i in range(1960,2018,1)])

df = pandas.read_excel('world-development-indicators.xlsx')
df = df[:792]
df.columns = column_name

In [2]:
df.sample(3)

,type,series,name,code,1960,1961,1962,1963,1964,1965,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
545,"Population, male (% of total)",SP.POP.TOTL.MA.ZS,Barbados,BRB,44.8002,44.8944,45.1085,45.3998,45.7083,45.9889,...,47.9944,47.991,47.9837,47.9743,47.9598,47.9472,47.9297,47.9131,47.8949,47.8771
387,"Population, female (% of total)",SP.POP.TOTL.FE.ZS,"Korea, Dem. People’s Rep.",PRK,53.7895,53.6805,53.5933,53.516,53.432,53.3315,...,51.1508,51.1436,51.1359,51.1275,51.1191,51.1108,51.1034,51.0973,51.0925,51.089
119,"Population, total",SP.POP.TOTL,Jordan,JOR,932257,973083,1009733,1049302,1101459,1172550,...,6489822,6821116,7182390,7574943,7992573,8413464,8809306,9159302,9455802,9702353


In [3]:
import rdflib
from rdflib import Literal, RDF, URIRef, Namespace

from rdflib.namespace import RDFS,RDF,XSD
MYNS = Namespace('http://dsci558.org/myfakenamespace#')
QB = Namespace('http://purl.org/linked-data/cube#')

In [4]:
g = rdflib.Graph()

# bind namespaces to prefixes to shorten the URIs for Turtle
g.bind('my_ns', MYNS)
g.bind('rdfs',RDFS)
g.bind('qb',QB)

In [5]:
from collections import defaultdict,namedtuple
year_list = [str(i) for i in range(1960,2018,1)]
database = defaultdict(list)
Entry = namedtuple('Entry',['year','code','type','data'])

for row in df.itertuples():
    country_code = row.code
    type = row.type[12:]

    if 'female' in type:
        type = 'female'
    elif 'male' in type:
        type = 'male'
    else:
        type = 'total'
    
    for index,year in enumerate(year_list):
        row_number = index + 5
        database[year+country_code].append(Entry(year, country_code, type, row[row_number]))
database['1960AFG']        

[Entry(year='1960', code='AFG', type='total', data=8996351),
 Entry(year='1960', code='AFG', type='female', data=48.3194782591104),
 Entry(year='1960', code='AFG', type='male', data=51.6805217408896)]

In [6]:
from tqdm import tqdm
observation_id = 0 

for key in tqdm(database):
    year = key[:4]
    code = key[4:]
    obs_dict = dict()
    for entry in database[year + code]:
        if entry.type == "total":
            obs_dict['total'] = entry.data
        elif entry.type == "female":
            obs_dict['female'] = entry.data
        elif entry.type == "male":
            obs_dict['male'] = entry.data
    
    observation_uri = MYNS[f'observation_{observation_id}']
    g.add((observation_uri, RDF.type, QB['Observation']))
    g.add((observation_uri, MYNS['refPeriod'], URIRef(f'http://reference.data.gov.uk/id/gregorian-interval/{year}-01-01T00:00:00/P3Y')))
    g.add((observation_uri, MYNS['measure_population'], Literal(obs_dict['total'], datatype=XSD.integer)))
    g.add((observation_uri, MYNS['measure_femalePopulation'], Literal(obs_dict['female'], datatype=XSD.float)))
    g.add((observation_uri, MYNS['measure_malePopulation'], Literal(obs_dict['male'], datatype=XSD.float)))
    g.add((observation_uri, MYNS['refArea'], MYNS[code]))
    g.add((observation_uri, QB['dataSet'], MYNS['world_development_indicators']))
    
    observation_id += 1


100%|██████████| 15312/15312 [00:04<00:00, 3771.65it/s]


In [7]:
country_name = df['name'].to_numpy()
country = {
    df['code'].to_numpy()[i] : country_name[i]
    for i in range(len(country_name)) 
}
for code,name in tqdm(country.items()):
    country_uri = URIRef(MYNS[code])
    g.add((country_uri, RDF.type, MYNS['refArea']))
    g.add((country_uri, RDFS.label, Literal(name)))

100%|██████████| 264/264 [00:00<00:00, 16347.72it/s]


In [8]:
g.serialize('testoutput.ttl', format="turtle")

In [9]:
dataset_text = """\n
# Data Set
@prefix dct:    <http://purl.org/dc/terms/> .
@prefix sdmx-subject:   <http://purl.org/linked-data/sdmx/2009/subject#> .

my_ns:world_development_indicators a qb:DataSet;
    dct:title   "World Development Indicators"@en ;
    rdfs:label  "World Development Indicators"@en ;
    dct:subject sdmx-subject:1.1 ; # Demographic and social statistics -> Population and migration
    qb:structure my_ns:population .
"""

with open('testoutput.ttl','a+') as f:
    f.write(dataset_text)